In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm 
import matplotlib.cm as cm

In [ ]:
## Environment ##
def update_n(n, z, e1):
    """
    Update the resource state based on extraction and environmental impact.

    Parameters:
    - n (float): Current resource state.
    - z (float): Extraction level.
    - e1 (float): Relative speed of environmental dynamics to strategic dynamics.

    Returns:
    float: Updated resource state.

    This function calculates the updated resource state based on the extraction level and environmental impact factor.
    """
    return n + e1 * (z - n)

## Heuristics and updating ##
def predict_zl(z_evolution, heuristic, b1, b2, b3):
    """
    Predict the next extraction level using different heuristics.

    Parameters:
    - z_evolution (numpy.ndarray): Array of past extraction levels.
    - heuristic (int): Heuristic type (0-3).
    - b1, b2, b3 (float): Heuristic parameters.

    Returns:
    float: Predicted extraction level.

    This function predicts the next extraction level based on past observations and the chosen heuristic.
    """
    t = len(z_evolution)
    if heuristic == 0: # adaptive expectation on zl (essentially exponential weights on all memories)
        expec = np.sum([z_evolution[i] * b1 * (1-b1) ** (t-i-1) for i in range(t)])
    elif heuristic == 1: # trend-chasing expectation on zl
        if t > 1:
            expec = z_evolution[-1] + b2 * (z_evolution[-1] - z_evolution[-2]) 
        else: 
            expec = z_evolution[-1]  # no trend to extrapolate so just previous observation
    elif heuristic == 2: # contrarian (trend-chasing) expectation on zl
        if t > 1:
            expec = z_evolution[-1] + b3 * (z_evolution[-1] - z_evolution[-2]) 
        else: 
            expec = z_evolution[-1]  # no trend to extrapolate so just previous observation
    else: # anchoring and adjustment expectation on zl
        if t > 1:
            expec = 0.5 * (np.mean(z_evolution) + z_evolution[-1]) + (z_evolution[-1] - z_evolution[-2]) 
        else: 
            expec = z_evolution[-1] # no trend to extrapolate so just anchor point
    return np.clip(expec, 0, 1)

def update_heuristic(old_utilities, previous_z, previous_expectations, current_heuristic, h_upd_chance, eta, choice_intensity):
    """
    Update the chosen heuristic for an agent based on utility and chance.

    Parameters:
    - old_utilities (numpy.ndarray): Array of utilities for different heuristics.
    - previous_z (float): Previous extraction level.
    - previous_expectations (numpy.ndarray): Array of past expectations.
    - current_heuristic (int): Current heuristic choice.
    - h_upd_chance (float): Probability of updating the heuristic.
    - eta (float): Importance of past utilities for current heuristic utility.
    - choice_intensity (float): Intensity of choice.

    Returns:
    Tuple[int, numpy.ndarray]: Updated heuristic choice and new utilities.
    """
    new_utilities = np.array([-(previous_z - previous_expectations[i]) ** 2 + eta * old_utilities[i] for i in range(len(previous_expectations))])
    draw1 = np.random.uniform()
    if draw1 < h_upd_chance: # only update h_upd_chance percent of the time
        sum = np.sum(np.exp(choice_intensity * new_utilities))
        draw2 = np.random.uniform()
        count = np.exp(choice_intensity * new_utilities[0])
        id = 0
        while draw2 >= count/sum:
            id += 1
            count += np.exp(choice_intensity * new_utilities[id])
        return id, new_utilities
    else:
        return current_heuristic, new_utilities
    
## Agent decision-making ##
def base_sigmoid(pd, degree, sigma):
    """
    Calculate the sigmoid function used in agent decision-making, used for the probability of extracting with high effort.

    Parameters:
    - pd (float): Payoff difference.
    - degree (float): Agent's degree in the network.
    - sigma (float): Sigmoid parameter.

    Returns:
    float: Sigmoid value.
    """
    beta = np.sqrt(degree) / sigma 
    return 1 / (1 + np.exp(-beta * pd))

def action(n, z_evolution, d0, a1l, a1h, a2l, a2h, a3l, a3h, P, b1, b2, b3, heuristic, degree, sigma):
    """
    Make a decision regarding extraction based on heuristics and payoffs.

    Parameters:
    - n (float): Current resource state.
    - z_evolution (numpy.ndarray): Array of past extraction levels.
    - d0 (float): Constant in the payoff function.
    - a1l, a1h, a2l, a2h, a3l, a3h (float): Parameters in the payoff function.
    - P (int): Number of agents in the network.
    - b1, b2, b3 (float): Heuristic parameters.
    - heuristic (int): Chosen heuristic.
    - degree (float): Agent's degree in the network.
    - sigma (float): Sigmoid parameter.

    Returns:
    Tuple[int, float, float]: Action (0 or 1), predicted extraction level, and payoff difference.
    """
    ze_t = predict_zl(z_evolution, heuristic, b1, b2, b3)
    pd = d0 + (a1h-a1l) * (ze_t * (P-1)/P) + (a2h - a2l) * n + (a3h - a3l) * n * ze_t * (P-1)/P - a1l/P - a3l * n / P 

    random_draw = np.random.uniform()
    cutoff = base_sigmoid(pd, degree, sigma)
    if random_draw < cutoff:
        return 1, ze_t, pd
    else:
        return 0, ze_t, pd
    
## Network functions ##
def gen_net(P, ne):
    """
    Generate an Erdös-Rényi or Barabasi-Albert network.

    Parameters:
    - P (int): Number of agents in the network.
    - ne: link density in ER network (float) or number of new links per new node in BA network (int)

    Returns:
    Tuple[numpy.ndarray, numpy.ndarray]: Adjacency matrix and degree sequence.
    """
    if type(ne) == np.int64 or type(ne) == int:
        network = nx.barabasi_albert_graph(P,int(ne))
    else:
        network = nx.gnp_random_graph(P, ne)
    A = nx.to_numpy_array(network) + np.eye(P)
    return A, np.sum(A, axis=0) 

def construct_perceived_weights(A, impact):
    """
    Construct perceived weights for environmental impact.

    Parameters:
    - A (numpy.ndarray): Adjacency matrix.
    - impact (float): inequality of environmental impact.

    Returns:
    numpy.ndarray: Perceived weights for environmental impact.
    """
    P = A.shape[0]
    SA = np.sum(A, axis=0)
    SA_power = SA ** impact
    SA_imp_matrix = np.tile(SA_power, (P, 1))
    
    if impact == 0:
        perceived_weights = A / np.tile(np.sum(A, axis=1), (P, 1)).T 
    else:
        perceived_weights = A * SA_imp_matrix / (np.sum((SA * A) ** impact, axis=1)[:, np.newaxis])
    
    return perceived_weights

## Other functions ##
def mean_conf(results, x):
    """
    Calculate mean and confidence interval for simulation results.

    Parameters:
    - results (numpy.ndarray): Simulation results.
    - x (int): Number of simulations.

    Returns:
    Tuple[numpy.ndarray, numpy.ndarray]: Mean and confidence interval.
    """
    mean = np.mean(results, axis=0)
    conf = 1.96 * np.std(results, axis=0) / np.sqrt(x)
    return mean, conf 

def br_longrun(d0, a1l, a1h, a2l, a2h, a3l, a3h, P):
    """
    Calculate the full-information equilibrium extraction level.

    Parameters:
    - d0, a1l, a1h, a2l, a2h, a3l, a3h (float): Parameters in the payoff function.
    - P (float): Number of agents.

    Returns:
    numpy.ndarray: Long-run equilibrium extraction level.

    This function calculates the long-run equilibrium extraction level based on the payoff function parameters.
    """
    if a3l == a3h:
        num = a1l / P - d0
        den = (a1h - a1l) * (P - 1) / P + a2h - a2l - a3l/P
        zlr = np.array([num/den, num/den])
    else:
        a = d0-a1l/P
        b = (a1h-a1l) * (P-1) + a2h-a2l - a3l/P
        c = (a3h-a3l) * (P-1)/P
        D = b ** 2 - 4 * a * c
        zlr = np.array([(-b + np.sqrt(D)) / (2*a), (-b - np.sqrt(D)) / (2*a)])
    np.put(zlr, np.append(np.where(zlr>1), np.where(zlr<0)),np.nan)
    return zlr

def calc_moment(degrees, P, n_bins=20, moment=3):
    """
    Calculate a moment of the degree distribution.

    Parameters:
    - degrees (numpy.ndarray): Degree sequence.
    - P (int): Number of agents in the network.
    - n_bins (int): Number of bins for histogram.
    - moment (int): Order of the moment to calculate.

    Returns:
    float: Calculated moment.
    """
    weights, bin_edges = np.histogram(degrees, bins=n_bins, range=(0,P))
    values = np.array([np.mean([bin_edges[i], bin_edges[i+1]]) for i in range(n_bins)])
    ave = np.average(values, weights=weights)
    var = np.sum(np.array([weights[i]*(values[i]-ave)**moment for i in range(n_bins)]))
    return var ** (1/moment) / P

## Model ##
def model(n, z, P, ne, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity):
    """
    Simulate the agent-based model.

    Parameters:
    - n (float): Initial resource state.
    - z (float): Initial extraction level.
    - P (int): Number of agents in the network.
    - ne: link density in ER network (float) or number of new links per new node in BA network (int).
    - T (int): Number of time steps.
    - e1 (float): Relative speed of environmental dynamics to strategic dynamics.
    - d0, a1l, a1h, a2l, a2h, a3l, a3h (float): Parameters in the payoff function.
    - impact (float): Inequality of environmental impact.
    - sigma (float): Sigmoid parameter.
    - b1, b2, b3 (float): Heuristic parameters.
    - h_upd_chance (float): Probability of updating the heuristic.
    - eta (float): Importance of past utilities for current heuristic utility.
    - choice_intensity (float): Intensity of choice.

    Returns:
    Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray, float]:
    - n_evolution (numpy.ndarray): Evolution of resource state.
    - z_evolution (numpy.ndarray): Evolution of extraction level.
    - z_predictions (numpy.ndarray): Predicted extraction levels.
    - pd (numpy.ndarray): Payoff differences.
    - heuristic_fraction (numpy.ndarray): Fraction of agents using each heuristic.
    - observed_z (numpy.ndarray): Observed extraction levels.
    - acting_p (numpy.ndarray): Index of acting agent.
    - degrees (numpy.ndarray): Degree sequence of the network.
    - skew (float): Skewness of the degree distribution.
    """
    z = 1 # with environmental weights, initial conditions for z are a bit tricky 
    A, degrees = gen_net(P, ne) # form the network
    skew = calc_moment(degrees, P)
    n_evolution = np.ones((T*P+1)) * n 
    z_evolution = np.ones((T*P+1)) * z       
    actions = np.random.permutation(np.append(np.zeros(int(z*P)), np.ones(P-int(z*P)))) 
    z_predictions = np.empty(T*P)
    observed_z = np.empty(T*P)
    pd = np.empty(T*P)
    heuristic = np.random.randint(low=0, high=4, size=P) # period 1 and 2 everyone uses a random heuristic; high 4 because 4 heuristics
    heuristic_fraction = np.zeros((T*P+1, 4))
    heuristic_fraction[0,:] = np.array([np.count_nonzero(heuristic == h)/P for h in range(4)])
    utilities = np.zeros((P, 4)) # four heuristics 
    memory = np.zeros((P,P)) * np.nan 
    indices = np.where(A == 1)
    memory[indices] = actions[indices[1]]
    perceived_z = {i: np.array([]) for i in range(P)}
    perceived_n = np.ones(P) * n
    acting_p = np.empty(T*P)
    weights = degrees ** impact / np.sum(degrees ** impact) # these are used to update the resource state
    perceived_z_weights = construct_perceived_weights(A - np.eye(P), impact) # these have no self link and are used for taking into account environmental impact for forming expectations
    perceived_n_weights = construct_perceived_weights(A, impact) # these have a self link and are used for taking into account environmental impact for resource state perception updates

    for i in range(T*P):
        # choose a random player that acts
        id = np.random.randint(0,P)
        acting_p[i] = id
        old_heuristic = heuristic[id]
        old_action = actions[id]
        observation = memory[id].copy()
        observation[id] = np.nan # since want to predict the other users' behaviour, not your own
        perceived_z[id] = np.append(perceived_z[id], 1-np.nansum(observation * perceived_z_weights[id,:])) # take a snapshot of behaviour each time you act
        observed_z[i] = perceived_z[id][-1]

        # choose heuristic
        if len(perceived_z[id]) > 1: # can't update heuristics in first two periods as requires two previous expectations
            first_order_expectations = np.array([predict_zl(perceived_z[id][:-1], h, b1, b2, b3) for h in range(4)])
            heuristic[id], utilities[id,:] = update_heuristic(utilities[id,:], perceived_z[id][-1], first_order_expectations, heuristic[id], 
                                                    h_upd_chance, eta, choice_intensity)

        # choose which action maximizes discounted expected payoff
        actions[id], z_predictions[i], pd[i] = action(perceived_n[id], perceived_z[id], d0, a1l, a1h, a2l, a2h, a3l, a3h, P, b1, b2, b3, heuristic[id], degrees[id], sigma)
        
        # update agents' memories and perceived_n
        if A[:, id].any():
            memory[:, id] = np.where(A[:, id], actions[id], memory[:, id])
        perceived_n = update_n(perceived_n, 1-np.nansum(memory * perceived_n_weights, axis=1), e1/P) # each time someone acts, update perception of n using your view of what everyone is doing

        # update z, environment, and heuristic fraction
        diff = actions[id] - old_action
        if diff == 1:
            z_evolution[i+1] = z_evolution[i] - weights[id] 
        elif diff == -1:
            z_evolution[i+1] = z_evolution[i] + weights[id] 
        else:
            z_evolution[i+1] = z_evolution[i]
        n_evolution[i+1] = update_n(n_evolution[i], z_evolution[i+1], e1/P)

        heuristic_fraction[i+1,:] = heuristic_fraction[i,:]
        heuristic_fraction[i+1, old_heuristic] -= 1/P 
        heuristic_fraction[i+1, heuristic[id]] += 1/P 
        
    return n_evolution, z_evolution, z_predictions, pd, heuristic_fraction, observed_z, acting_p, degrees, skew

## Run model many times ##
def run_model(x, n, z, P, ne, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity):
    """
    Run the agent-based model multiple times and aggregates the results for analysis.

    Parameters:
    - x (int): Number of simulations.
    - n (float): Initial resource state.
    - z (float): Initial extraction level.
    - P (int): Number of agents in the network.
    - ne: link density in ER network (float) or number of new links per new node in BA network (int).
    - T (int): Number of time steps.
    - e1 (float): Relative speed of environmental dynamics to strategic dynamics.
    - d0, a1l, a1h, a2l, a2h, a3l, a3h (float): Parameters in the payoff function.
    - impact (float): Inequality of environmental impact.
    - sigma (float): Sigmoid parameter.
    - b1, b2, b3 (float): Heuristic parameters.
    - h_upd_chance (float): Probability of updating the heuristic.
    - eta (float): Importance of past utilities for current heuristic utility.
    - choice_intensity (float): Intensity of choice.

    Returns:
    Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray]:
    - n_results (numpy.ndarray): Evolution of resource state for each simulation.
    - pd_results (numpy.ndarray): Payoff differences for each simulation.
    - acting_p_results (numpy.ndarray): Index of acting agent for each simulation.
    - degrees_results (numpy.ndarray): Degree sequences for each simulation.
    - heuristic_results (numpy.ndarray): Fraction of agents using each heuristic for each simulation.
    """
    n_results = np.zeros((x, T*P+1))
    pd_results = np.zeros((x, T*P))
    acting_p_results = np.zeros((x, T*P))
    degrees_results = np.zeros((x, P))
    heuristic_results = np.zeros((x, T*P+1, 4))

    for i in tqdm(range(x)):
        n_evolution, z_evolution, zl_predictions, pd, heuristic_fraction, observed_z, acting_p, degrees, skew \
            = model(n, z, P, ne, T, e1, d0, a1l, a1h, a2l, a2h, a3l, a3h, impact, sigma, b1, b2, b3, h_upd_chance, eta, choice_intensity)
        n_results[i,:] = n_evolution 
        pd_results[i,:] = pd 
        acting_p_results[i,:] = acting_p  
        degrees_results[i,:] = degrees
        heuristic_results[i,:,:] = heuristic_fraction
    
    return n_results, pd_results, acting_p_results, degrees_results, heuristic_results